In [1]:
import torch
torch.cuda.empty_cache()


In [2]:
import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union, Literal, ForwardRef
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json, gen
import torch, outlines
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json
import guidance
from utils import timing_decorator

In [3]:

with open("../output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md", "r") as f:
    markdown_content = f.read()


In [4]:

MODEL_ID = "Qwen/Qwen3-30B-A3B"

hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",               
    low_cpu_mem_usage=True,          
)


tok        = AutoTokenizer.from_pretrained(MODEL_ID)


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:

model = guidance.models.Transformers(hf_model, tok)

gpustat is not installed, run `pip install gpustat` to collect GPU stats.


In [1]:
from pydantic import BaseModel, Field
from typing import List, Any

class DomainVariable(BaseModel):
    """Defines a single variable the LLM can use in the computation graph."""
    name: str = Field(..., description="The unique identifier for the variable.")
    description: str = Field(..., description="A detailed explanation of what this variable represents.")
    # Optional: You could add type hints, units, etc. for more advanced validation
    unit: Optional[str] = Field(..., description="The unit of the variable")
    data_type : type = Field(..., description="The data type of the variable")


NameError: name 'Optional' is not defined

In [14]:
# 1. Master registry of ALL possible variables
ALL_VARIABLES = {
    'transfer_passenger_count': DomainVariable(name='transfer_passenger_count', description='Total number of transferring passengers.', unit = None, data_type=float),
    'airline_scheduling_season': DomainVariable(name='airline_scheduling_season', description='Whether summer / winter airline scheduling season.', unit = None, data_type=float),
    'takeoff_aircraft_mtow_tonnes': DomainVariable(name='takeoff_aircraft_mtow_tonnes', description='The Maximum Take-Off Weight in tonnes.', unit = 'tonne', data_type=float),
    'landing_aircraft_mtow_tonnes': DomainVariable(name='landing_aircraft_mtow_tonnes', description='The Maximum Landing Weight in tonnes.', unit = 'tonne', data_type=float)
    
}

# 2. Map charge types to the variable names they are allowed to use
CHARGE_CATEGORY_VARIABLES = {
    "transfer_passenger_charge": [
        'transfer_passenger_count', 
        'airline_scheduling_season'
    ],
    "runway_landing_charge": [
        'landing_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],
    "runway_takeoff_charge": [
        'takeoff_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],
}

In [7]:
def create_dynamic_variable_enum(charge_category: str) -> type(Enum):
    """
    Creates a new Enum class containing only the variables relevant
    to the specified charge category.
    """
    variable_names = CHARGE_CATEGORY_VARIABLES.get(charge_category)
    if not variable_names:
        raise ValueError(f"Unknown charge category: {charge_category}")
    
    # The dictionary for the Enum must have {MEMBER_NAME: value}
    # We'll use uppercase for the member name for convention.
    enum_dict = {name.upper(): name for name in variable_names}
    
    # Create the Enum class dynamically
    return Enum("Var", enum_dict)

Var = create_dynamic_variable_enum("transfer_passenger_charge")
print(Var.TRANSFER_PASSENGER_COUNT.value)

NameError: name 'CHARGE_CATEGORY_VARIABLES' is not defined

In [30]:
allowed_variables = [el.value for el in list(Var)]
allowed_variables

['transfer_passenger_count', 'airline_scheduling_season']

In [22]:
# class VariableNode(BaseModel):
#     """A leaf node representing a symbolic input variable."""
#     type: Literal["VARIABLE"] = "VARIABLE"
#     # The 'name' will be constrained to a dynamic Enum of allowed variables
#     name: Var 
#     description: str = Field(..., description="Explanation of what input this variable represents.")

# class Op(str, Enum):
#     ADD = "ADD"
#     MULTIPLY = "MULTIPLY"

# class ValueNode(BaseModel):
#     """A leaf node representing a fixed numeric constant."""
#     type: Literal["VALUE"] = "VALUE"
#     value: float
#     description: str = Field(description="Explanation of what this value represents")
#     # unit: 

# class OpNode(BaseModel):
#     """An operation node with two children"""
#     type: Literal["OPERATION"] = "OPERATION"
#     operator: Op
#     # The 'left' and 'right' fields will be added dynamically

# # the Union for the recursive fields
# AnyNode = Union[OpNode, ValueNode, VariableNode]

# # Dynamically update the OpNode to add the recursive fields
# OpNode.model_fields.update({
#     'left': (AnyNode, Field(..., discriminator='type')),
#     'right': (AnyNode, Field(..., discriminator='type')),
# })

# # The root of the expression tree must be an operation.
# class Node(RootModel):
#     """The root of the expression tree, which must be an OpNode."""
#     root: OpNode


In [76]:
from pydantic import BaseModel, Field, RootModel
from typing import Union, Literal
from enum import Enum

# Separate enums for clarity and type safety
class MathOperator(str, Enum):
    ADD = "ADD"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"

class Comparator(str, Enum):
    GREATER_THAN = "GREATER_THAN"
    LESS_THAN = "LESS_THAN"
    EQUAL_TO = "EQUAL_TO"

# --- Node Definitions ---

class ValueNode(BaseModel):
    type: Literal["VALUE"] = "VALUE"
    value: float
    description: str

class VariableNode(BaseModel):
    type: Literal["VARIABLE"] = "VARIABLE"
    name: str 
    description: str

class BinaryOpNode(BaseModel):
    """Node for mathematical operations that produce a number."""
    type: Literal["BINARY_OPERATION"] = "BINARY_OPERATION"
    operator: MathOperator
    left: 'AnyNode'
    right: 'AnyNode'

class ComparisonNode(BaseModel):
    """Node for comparison operations that produce a boolean."""
    type: Literal["COMPARISON"] = "COMPARISON"
    operator: Comparator
    left: 'AnyNode'
    right: 'AnyNode'

class ConditionalNode(BaseModel):
    """Node for if-then-else logic."""
    type: Literal["CONDITIONAL"] = "CONDITIONAL"
    condition: ComparisonNode # Condition must be a comparison
    if_true: 'AnyNode'
    if_false: 'AnyNode'

# --- Recursive Setup ---

AnyNode = Union[
    ValueNode, 
    VariableNode, 
    BinaryOpNode, 
    ConditionalNode
]

# Use model_rebuild() to safely resolve all forward references
BinaryOpNode.model_rebuild()
ConditionalNode.model_rebuild()
ComparisonNode.model_rebuild()

class Node(RootModel):
    root: BinaryOpNode

In [95]:

@guidance
def create_expression_tree(llm, allowed_variables_prompt, document, query, output_schema):
    
    with system():
        llm = llm + f"""You are a world-class algorithm for building expression trees from text. Your goal is to construct a JSON object that represents the calculation logic for a 'charge' based on a document and a set of conditions.
        
You MUST follow the provided JSON schema exactly. The graph has three types of nodes:
1.  `OpNode`: For mathematical operations. The `operator` must be 'ADD' or 'MULTIPLY'.
2.  `ValueNode`: Use this as a LEAF NODE AND ONLY for FIXED NUMERIC CONSTANTS found directly in the document (e.g., a fee of £3.90, a rate of 1.5%).
3.  `VariableNode`: Use this as a LEAF NODE AND ONLY for SYMBOLIC INPUTS required by the formula. The `name` of the variable MUST be chosen from the 'Allowed Variables' list below.


**Allowed Variables for this Task:**
---
{allowed_variables_prompt}
---
""" + """

Here is an example of valid expression trees:
1. Simple multiplication: {"type": "OPERATION", "operator": "MULTIPLY", "left": {"type": "VALUE", "value": 3.9, "description": "Transfer passenger charge per passenger"}, "right": {"type": "VARIABLE", "description": "Number of transfer passengers"}}
"""

    with user():
        llm += f"""Here is the document:
---
{document}
---

**Query:**
Based on the document, construct the computation graph for the following request:
"{query}"
"""
    with assistant():
        llm += gen_json(
            name="expression_tree", 
            schema=output_schema,
            max_tokens=1000,
        )

    return llm


In [34]:
# allowed_variables_prompt = "\n".join(
#             [f"- **{v.name}**: {v.description}" for name, v in ALL_VARIABLES.items() if name in allowed_variables]
# )

# allowed_variables_prompt

'- **transfer_passenger_count**: Total number of transferring passengers.\n- **airline_scheduling_season**: Whether summer / winter airline scheduling season.'

In [96]:

class ComputationGraphBuilder:
    """
    Orchestrates the creation of a computation graph by preparing dynamic
    constraints and prompting the LLM.
    """
    
    def __init__(self, model):
        """
        Initializes the builder with a guidance model.
        """
        self.model = model
        # Set the default LLM for all guidance programs
        # guidance.llm = self.model
    @timing_decorator
    def build(self, document_content: str, query: str, charge_category: str) -> dict:
        """
        Generates a computation graph for a given query and document.

        Args:
            document_content: The text containing the rules.
            query: A natural language question about what to calculate.
            charge_category: The specific charge context used to filter variables.

        Returns:
            A dictionary representing the computation graph or an error.
        """
        print(f"--- Building graph for charge category: '{charge_category}' ---")
        
        # 1. Dynamically create the filtered Enum for this specific task
        try:
            Var = create_dynamic_variable_enum(charge_category)
        except ValueError as e:
            print(f"Error: {e}")
            return {"error": str(e)}

        # 3. Create a formatted prompt string of allowed variables for the LLM
        allowed_variables = [el.value for el in list(Var)]
        allowed_variables_prompt = "\n".join(
            [f"- **{v.name}**: {v.description}" for name, v in ALL_VARIABLES.items() if name in allowed_variables]
        )

        try:
            # 4. Execute the guidance program with all dynamic components
            result_lm = self.model + create_expression_tree(
                allowed_variables_prompt=allowed_variables_prompt,
                document=document_content,
                query=query,
                output_schema=Node
            )
            
            pydantic_graph = result_lm["expression_tree"]
            # print("\nSuccessfully generated graph:")
            # # Use model_dump_json for Pydantic v2
            # print(pydantic_graph.model_dump_json(indent=2)) 
            return pydantic_graph
            
        except Exception as e:
            print(f"\nAn error occurred while building the graph for '{query}': {e}")
            return {"error": str(e)}

In [45]:
# 1. Instantiate the builder
graph_builder = ComputationGraphBuilder(model=model)

# 2. Define a query and its context
my_query = "Calculate the total transfer passenger charge for the summer scheduling season."
my_charge_category = "transfer_passenger_charge"

# 3. Call the build method to generate the graph
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)

--- Building graph for charge category: 'transfer_passenger_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [46]:
expression_tree_dict

'{"type": "OPERATION", "operator": "MULTIPLY", "left": {"type": "VALUE", "value": 3.9, "description": "Transfer passenger charge per passenger"}, "right": {"type": "VARIABLE", "name": "transfer_passenger_count", "description": "Number of transfer passengers"}, "description": "Total transfer passenger charge for summer scheduling season."}'

In [49]:
loaded_json = json.loads(expression_tree_dict)

In [51]:
loaded_json

{'type': 'OPERATION',
 'operator': 'MULTIPLY',
 'left': {'type': 'VALUE',
  'value': 3.9,
  'description': 'Transfer passenger charge per passenger'},
 'right': {'type': 'VARIABLE',
  'name': 'transfer_passenger_count',
  'description': 'Number of transfer passengers'},
 'description': 'Total transfer passenger charge for summer scheduling season.'}

In [53]:
print(json.dumps(loaded_json, indent=4))

{
    "type": "OPERATION",
    "operator": "MULTIPLY",
    "left": {
        "type": "VALUE",
        "value": 3.9,
        "description": "Transfer passenger charge per passenger"
    },
    "right": {
        "type": "VARIABLE",
        "name": "transfer_passenger_count",
        "description": "Number of transfer passengers"
    },
    "description": "Total transfer passenger charge for summer scheduling season."
}


In [56]:
result = Node.model_validate_json(expression_tree_dict)

In [58]:
print(result.model_dump_json(indent=8))

{
        "type": "OPERATION",
        "operator": "MULTIPLY"
}


In [59]:
import sympy

def json_to_sympy_expr(node: dict):
    """
    Recursively converts a JSON expression tree node into a Sympy expression.
    """
    node_type = node.get('type')

    if node_type == 'OPERATION':
        left_expr = json_to_sympy_expr(node['left'])
        right_expr = json_to_sympy_expr(node['right'])
        
        if node['operator'] == 'MULTIPLY':
            return left_expr * right_expr
        elif node['operator'] == 'ADD':
            return left_expr + right_expr
        else:
            raise ValueError(f"Unsupported operator: {node.get('operator')}")

    elif node_type == 'VALUE':
        return sympy.Number(node['value'])

    elif node_type == 'VARIABLE':
        return sympy.Symbol(node['name'])
        
    raise ValueError(f"Unknown node type: {node_type}")

In [61]:
loaded_json

{'type': 'OPERATION',
 'operator': 'MULTIPLY',
 'left': {'type': 'VALUE',
  'value': 3.9,
  'description': 'Transfer passenger charge per passenger'},
 'right': {'type': 'VARIABLE',
  'name': 'transfer_passenger_count',
  'description': 'Number of transfer passengers'},
 'description': 'Total transfer passenger charge for summer scheduling season.'}

In [62]:
expression = json_to_sympy_expr(loaded_json)
print(f"Generated Sympy Expression: {expression}")

# 3. Evaluate the expression by substituting values for the variables
variables = {'transfer_passenger_count': 30}
result = expression.subs(variables)

print(f"Result of substituting {variables}: {result}")

Generated Sympy Expression: 3.9*transfer_passenger_count
Result of substituting {'transfer_passenger_count': 30}: 117.000000000000


In [65]:

my_query = "Calculate the total transfer passenger charge for the winter scheduling season."
my_charge_category = "transfer_passenger_charge"

# 3. Call the build method to generate the graph
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)

--- Building graph for charge category: 'transfer_passenger_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [66]:
loaded_json = json.loads(expression_tree_dict)
loaded_json

{'type': 'OPERATION',
 'operator': 'MULTIPLY',
 'left': {'type': 'VALUE',
  'value': 2.8,
  'description': 'Transfer passenger charge per passenger'},
 'right': {'type': 'VARIABLE',
  'name': 'transfer_passenger_count',
  'description': 'Number of transfer passengers'},
 'description': 'Total transfer passenger charge for winter scheduling season.'}

In [67]:
print(json.dumps(loaded_json, indent=4))

{
    "type": "OPERATION",
    "operator": "MULTIPLY",
    "left": {
        "type": "VALUE",
        "value": 2.8,
        "description": "Transfer passenger charge per passenger"
    },
    "right": {
        "type": "VARIABLE",
        "name": "transfer_passenger_count",
        "description": "Number of transfer passengers"
    },
    "description": "Total transfer passenger charge for winter scheduling season."
}


In [ ]:

my_query = "Calculate the total transfer passenger charge for the winter scheduling season."
my_charge_category = "transfer_passenger_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

In [73]:
execution_time

232.55998749099672

In [78]:
ALL_VARIABLES = {
    
    'transfer_passenger_count': DomainVariable(name='transfer_passenger_count', description='Total number of transferring passengers.', unit=None, data_type=float),
    'airline_scheduling_season': DomainVariable(name='airline_scheduling_season', description='Whether summer/winter airline scheduling season.', unit=None, data_type=float),
    'takeoff_aircraft_mtow_tonnes': DomainVariable(name='takeoff_aircraft_mtow_tonnes', description='The Maximum Take-Off Weight in tonnes.', unit='tonne', data_type=float),
    'landing_aircraft_mtow_tonnes': DomainVariable(name='landing_aircraft_mtow_tonnes', description='The Maximum Landing Weight in tonnes.', unit='tonne', data_type=float),
    
    'parking_duration_hours': DomainVariable(
        name='parking_duration_hours',
        description='Total duration of parking in hours. Surcharges apply at 48 and 72 hours.',
        unit='hours',
        data_type=float
    ),
    'aircraft_stand_type': DomainVariable(
        name='aircraft_stand_type',
        description='The type of aircraft stand used for parking. E.g., "Wide Contact", "Narrow Remote", "LAP", "Long Term Remote".',
        unit=None,
        data_type=str
    ),
    'parking_location': DomainVariable(
        name='parking_location',
        description='The location of the parking stand, either "EAP" (East Aerodrome Parking) or "WAP" (West Aerodrome Parking).',
        unit=None,
        data_type=str
    ),
    'is_overnight_parking': DomainVariable(
        name='is_overnight_parking',
        description='True if the parking occurs during the free overnight period (2300-0600hrs).',
        unit=None,
        data_type=bool
    )
}
# 2. Map charge types to the variable names they are allowed to use
CHARGE_CATEGORY_VARIABLES = {
    # --- Existing Categories ---
    "transfer_passenger_charge": [
        'transfer_passenger_count', 
        'airline_scheduling_season'
    ],
    "runway_landing_charge": [
        'landing_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],
    "runway_takeoff_charge": [
        'takeoff_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],

    # --- New Categories for Parking Charges ---
    "east_aerodrome_parking_charge": [
        'parking_duration_hours',
        'aircraft_stand_type',
        'is_overnight_parking'
        # 'parking_location' is implicitly 'EAP' for this category
    ],
    "west_aerodrome_parking_charge": [
        'parking_duration_hours',
        'aircraft_stand_type',
        'is_overnight_parking'
        # 'parking_location' is implicitly 'WAP' for this category
    ]
}



In [97]:

my_query = "Calculate the total parking charge for a wide remote aircraft stand type at West Aerodrome Parking with no overnight parking."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [99]:
execution_time

153.85250417119823

In [100]:
loaded_json = json.loads(expression_tree_dict)
print(json.dumps(loaded_json, indent=4))

{
    "type": "VARIABLE",
    "name": "parking_duration_hours",
    "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours."
}


In [104]:
@guidance
def create_graph_with_cot(llm, allowed_variables_prompt, document, query, output_schema):
    
    with system():
        llm += f"""You are an expert system that converts textual calculation rules into structured JSON expression trees.
        You MUST follow a step-by-step reasoning process before generating the final JSON.

        **Reasoning Steps:**
        1.  **Identify Base Rate:** Find the base numerical rate for the charge described in the user's query from the document.
        2.  **Identify Variables:** List the symbolic variables needed for the calculation (e.g., parking_duration_hours).
        3.  **Identify Logic:** Find any conditional logic (if-then-else surcharges) or mathematical conversions (e.g., hours to days).
        4.  **Synthesize Plan:** Briefly describe how you will combine these pieces into a final expression tree.

        After writing your reasoning, you will generate the JSON object.

        **Allowed Variables for this Task:**
        ---
        {domain_variables_prompt}
        ---
        """

    with user():
        llm += f"""
        **Document:**
        ---
        {document}
        ---

        **Query:**
        Based on the document, construct the computation graph for the following request:
        "{query}"
        """

    with assistant():
        # This is the new Chain of Thought part. The model must "think" first.
        llm += "My step-by-step thinking process:\n"
        llm += gen("thought", stop="\n\n", max_tokens=400)
        
        # After thinking, it generates the JSON.
        llm += "\n\nFinal JSON object:\n"
        llm += gen_json(
            name="result_graph", 
            schema=output_schema,
            max_tokens=1000 
        )
        
    return llm

In [115]:

class ComputationGraphBuilder:
    """
    Orchestrates the creation of a computation graph by preparing dynamic
    constraints and prompting the LLM.
    """
    
    def __init__(self, model):
        """
        Initializes the builder with a guidance model.
        """
        self.model = model
        # Set the default LLM for all guidance programs
        # guidance.llm = self.model
    @timing_decorator
    def build(self, document_content: str, query: str, charge_category: str) -> dict:
        """
        Generates a computation graph for a given query and document.

        Args:
            document_content: The text containing the rules.
            query: A natural language question about what to calculate.
            charge_category: The specific charge context used to filter variables.

        Returns:
            A dictionary representing the computation graph or an error.
        """
        print(f"--- Building graph for charge category: '{charge_category}' ---")
        
        # 1. Dynamically create the filtered Enum for this specific task
        try:
            Var = create_dynamic_variable_enum(charge_category)
        except ValueError as e:
            print(f"Error: {e}")
            return {"error": str(e)}

        # 3. Create a formatted prompt string of allowed variables for the LLM
        allowed_variables = [el.value for el in list(Var)]
        allowed_variables_prompt = "\n".join(
            [f"- **{v.name}**: {v.description}" for name, v in ALL_VARIABLES.items() if name in allowed_variables]
        )

        try:
            # 4. Execute the guidance program with all dynamic components
            result_lm = self.model + create_graph_with_cot(
                allowed_variables_prompt=allowed_variables_prompt,
                document=document_content,
                query=query,
                output_schema=Node
            )
            
            
            # print("\nSuccessfully generated graph:")
            # # Use model_dump_json for Pydantic v2
            # print(pydantic_graph.model_dump_json(indent=2)) 
            return result_lm
            
        except Exception as e:
            print(f"\nAn error occurred while building the graph for '{query}': {e}")
            return {"error": str(e)}

In [116]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a wide remote aircraft stand type at West Aerodrome Parking with no overnight parking."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [117]:
execution_time

333.471676080022

In [123]:
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))  

{
    "type": "COMPARISON",
    "operator": "EQUAL_TO",
    "left": {
        "type": "VARIABLE",
        "name": "parking_stand_type",
        "description": "Type of aircraft stand (e.g., wide remote, narrow remote, etc.)"
    },
    "right": {
        "type": "VALUE",
        "value": 9.6,
        "description": "Base rate for wide remote stand at WAP"
    }
}


In [122]:
expression_tree_dict['result_graph']

'{"type": "COMPARISON", "operator": "EQUAL_TO", "left": {"type": "VARIABLE", "name": "parking_stand_type", "description": "Type of aircraft stand (e.g., wide remote, narrow remote, etc.)"}, "right": {"type": "VALUE", "value": 9.60, "description": "Base rate for wide remote stand at WAP"}}'